Data scientists commonly spend over half their time cleaning data, so knowing how to clean "messy" data is an extremely important skill.

We'll learn the basics of data cleaning with pandas as we work with `laptops.csv`, a CSV file containing information about 1,300 laptop computers

In [2]:
import pandas as pd

laptops = pd.read_csv("laptops.csv",)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 4: invalid continuation byte

We get an error! (The error message has been shortened.) This error references UTF-8, which is a type of encoding. Computers, at their lowest levels, can only understand binary 0 & 1 and encodings are systems for representing characters in binary.

Something we can do if our file has an unknown encoding is to try the most common encodings:

* `UTF-8`
* `Latin-1` (also known as ISO-8895-1)
* `Windows-1251`

In [3]:
laptops = pd.read_csv("laptops.csv", encoding = "latin-1")
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
 Storage                    1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 132.4+ KB


We can see that every column is represented as the **object** type, indicating that they are represented by `strings`, not numbers. Also, one of the columns, `Operating System Version`, has null values.

The column labels have a variety of upper and lowercase letters, as well as spaces and parentheses, which will make them harder to work with and read. One noticeable issue is that the `"Storage"` column name has a space in front of it. These quirks with column labels can sometimes be hard to spot, so removing extra whitespaces from all column names will save us more work in the long run.

Remove any whitespace from the start and end of each column name

In [4]:
new_columns = []

for c in laptops.columns:
    clean_c = c.strip()
    new_columns.append(clean_c)

laptops.columns = new_columns
laptops.columns

Index(['Manufacturer', 'Model Name', 'Category', 'Screen Size', 'Screen',
       'CPU', 'RAM', 'Storage', 'GPU', 'Operating System',
       'Operating System Version', 'Weight', 'Price (Euros)'],
      dtype='object')

Column labels still have a variety of upper and lowercase letters, as well as parentheses, which will make them harder to work with and read.

Cleaning our column labels by:

* Replacing spaces with underscores.
* Removing special characters.
* Making all labels lowercase.
* Shortening any long column names.

In [21]:
def clean_col(c):
    c = c.strip()
    c = c.replace("(","")
    c = c.replace(")","")
    c = c.lower()
    c = c.replace("operating_system", "os")
    c = c.replace(" ", "_")   
    return c

In [22]:
clean_columns = [clean_col(c) for c in laptops.columns]
laptops.columns = clean_columns
laptops.columns

Index(['manufacturer', 'model_name', 'category', 'screen_size_inches',
       'screen', 'cpu', 'ram_gb', 'storage', 'gpu', 'os', 'os_version',
       'weight', 'price_euros', 'gpu_manufacturer', 'cpu_manufacturer'],
      dtype='object')

In [7]:
print(laptops["screen_size"].dtype)
print(laptops["screen_size"].unique())

object
['13.3"' '15.6"' '15.4"' '14.0"' '12.0"' '11.6"' '17.3"' '10.1"' '13.5"'
 '12.5"' '13.0"' '18.4"' '13.9"' '12.3"' '17.0"' '15.0"' '14.1"' '11.3"']


# Convert the screen_size column to numeric

In [8]:
clean_screen = []
for i in laptops["screen_size"]:
    clean = i.strip('"')
    clean = float(clean)
    clean_screen.append(clean)

In [9]:
laptops["screen_size"] = clean_screen
laptops["screen_size"].dtype

dtype('float64')

# Evaluating ram

In [10]:
unique_ram = laptops["ram"].unique()
unique_ram

array(['8GB', '16GB', '4GB', '2GB', '12GB', '6GB', '32GB', '24GB', '64GB'],
      dtype=object)

The pandas library contains dozens of vectorized string methods we can use to manipulate text data, many of which perform the same operations as Python string methods. Most vectorized string methods are available using the Series.str accessor, which means we can access them by adding `str` between the series name and the method name:

We can use the `Series.str.replace()` method, which is a vectorized version of the Python `str.replace()`

In [11]:
laptops["ram"] = laptops["ram"].str.replace("GB","").astype(int)


In [12]:
laptops["ram"].unique()

array([ 8, 16,  4,  2, 12,  6, 32, 24, 64], dtype=int64)

In [13]:
laptops.dtypes

manufacturer                 object
model_name                   object
category                     object
screen_size                 float64
screen                       object
cpu                          object
ram                           int32
storage                      object
gpu                          object
operating_system             object
operating_system_version     object
weight                       object
price_euros                  object
dtype: object

# Changing columns Name

In [14]:
laptops.rename({"screen_size":"screen_size_inches","ram":"ram_gb"}, axis = 1, inplace=True)
#  we specify the axis=1 parameter so pandas knows that we want to rename labels in the column axis.
# we can either use inplace=True or assign the result back to the dataframe - both will give us the same results
laptops.columns

Index(['manufacturer', 'model_name', 'category', 'screen_size_inches',
       'screen', 'cpu', 'ram_gb', 'storage', 'gpu', 'operating_system',
       'operating_system_version', 'weight', 'price_euros'],
      dtype='object')

In [15]:
ram_gb_desc = laptops["ram_gb"].describe()
ram_gb_desc

count    1303.000000
mean        8.382195
std         5.084665
min         2.000000
25%         4.000000
50%         8.000000
75%         8.000000
max        64.000000
Name: ram_gb, dtype: float64

# Analysing columns (gpu & cpu)

In [16]:
laptops["gpu"].head(3)

0    Intel Iris Plus Graphics 640
1          Intel HD Graphics 6000
2           Intel HD Graphics 620
Name: gpu, dtype: object

The information in this column seems to be a manufacturer (Intel, AMD) followed by a model name/number. Let's extract the manufacturer by itself so we can find the most common ones

In [17]:
# extractinh the manufacturer name from the gpu column, and assigned it to a new column

laptops["gpu_manufacturer"] = laptops["gpu"].str.split(n= 1, expand = True)[0] # we can also use .iloc[:,0]
laptops["gpu_manufacturer"].head()

0    Intel
1    Intel
2    Intel
3      AMD
4    Intel
Name: gpu_manufacturer, dtype: object

In [18]:
laptops["cpu_manufacturer"] = laptops["cpu"].str.split(n=1).str[0] # .str[0] is used as alternate of abvoe method
laptops["cpu_manufacturer"].head()

0    Intel
1    Intel
2    Intel
3    Intel
4    Intel
Name: cpu_manufacturer, dtype: object

In [19]:
# The counts of each manufacturer in cpu_manufacturer
cpu_manufacturer_counts = laptops["cpu_manufacturer"].value_counts()
cpu_manufacturer_counts 

Intel      1240
AMD          62
Samsung       1
Name: cpu_manufacturer, dtype: int64

If our data has been scraped from a webpage or if there was manual data entry involved at some point, we may end up with inconsistent values. Let's look at an example from our `os column`:

In [23]:
laptops["os"].unique()

array(['macOS', 'No OS', 'Windows', 'Mac OS', 'Linux', 'Android',
       'Chrome OS'], dtype=object)

We can see that there are two variations of the Apple operating system — `macOS` — in our data set: `Mac OS` and `macOS`. One way we can fix this is with the `Series.map()` method. The `Series.map()` method is ideal when we want to change multiple values in a column.

The most common way to use `Series.map()` is with a **dictionary**

One important thing to remember with `Series.map()` is that if a value from our series doesn't exist as a key in our dictionary, it will convert that value to `NaN`.

In [24]:
map_dict =dict((('macOS','macOS'), ('No OS','No OS'),
                ('Windows','Windows'), ('Mac OS','macOS'),
                ('Linux','Linux'),('Android','Android'),
                ('Chrome OS','Chrome OS')))
map_dict

{'macOS': 'macOS',
 'No OS': 'No OS',
 'Windows': 'Windows',
 'Mac OS': 'macOS',
 'Linux': 'Linux',
 'Android': 'Android',
 'Chrome OS': 'Chrome OS'}

In [25]:
laptops["os"] = laptops["os"].map(map_dict)
laptops["os"].unique()

array(['macOS', 'No OS', 'Windows', 'Linux', 'Android', 'Chrome OS'],
      dtype=object)

In [1]:
# Alternate method for changing values

#laptops.loc[laptops["os"] == "'Mac OS'","os"] = "macOS"
# laptops["os"].str.replace('Mac OS',"macOS")


We've talked briefly about missing values and how both NumPy and pandas represent these as null values. In pandas, null values will be indicated by either `NaN` or `None`

We can use the `DataFrame.isnull()` method to identify missing values, which returns a boolean dataframe. We can then use the `DataFrame.sum()` method to give us a count of the `True` values for each column:

In [27]:
laptops.isnull().sum()

manufacturer            0
model_name              0
category                0
screen_size_inches      0
screen                  0
cpu                     0
ram_gb                  0
storage                 0
gpu                     0
os                      0
os_version            170
weight                  0
price_euros             0
gpu_manufacturer        0
cpu_manufacturer        0
dtype: int64

It's now clear that we have only one column with null values, `os_version`, which has 170 missing values.

There are a few options for handling missing values:

* Remove any rows that have missing values.
* Remove any columns that have missing values.
* Fill the missing values with some other value.
* Leave the missing values as it is.

The first two options are often used to prepare data for machine learning algorithms, which are unable to be used with data that includes null values. We can use the `DataFrame.dropna()` method to remove or drop rows and columns with null values.



The `DataFrame.dropna()` method accepts an axis parameter, which indicates whether we want to drop along the column or index axis.

* for columns axis, `axis = 1`

* for index axis, `axis = 0` (default value of dropna)

In [28]:
laptops_no_null_rows = laptops.dropna() # it will make copy of laptops DataFrame so it will not change original laptops DataFrame
laptops_no_null_cols = laptops.dropna(axis = 1)
laptops.isnull().sum()

manufacturer            0
model_name              0
category                0
screen_size_inches      0
screen                  0
cpu                     0
ram_gb                  0
storage                 0
gpu                     0
os                      0
os_version            170
weight                  0
price_euros             0
gpu_manufacturer        0
cpu_manufacturer        0
dtype: int64

While dropping rows or columns is the easiest approach to deal with missing values, it may not always be the best approach. For example, removing a disproportionate amount of one manufacturer's laptops could change our analysis.

In [30]:
laptops['os_version'].value_counts(dropna = False)

10      1072
NaN      170
7         45
X          8
10 S       8
Name: os_version, dtype: int64

In [31]:
# Let's also explore the os column, since it's is closely related to the os_version column. 

os_with_null_v = laptops.loc[laptops["os_version"].isnull(),"os"]
os_with_null_v.value_counts()

No OS        66
Linux        62
Chrome OS    27
macOS        13
Android       2
Name: os, dtype: int64

Immediately, we can observe a few things:

* The most frequent value is **"No OS"**. This is important to note because if there is no os, there shouldn't be a version defined in the `os_version` column.
* Thirteen of the laptops that come with **macOS** do not specify the version. We can use our knowledge of MacOS to confirm that `os_version` should be equal to `X`.

In [32]:
laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"
laptops.loc[laptops["os"] == "No OS", "os_version"] = "Version Unknown"

In [ ]:
# alternate to above

# laptops["os_version"] = laptops["os_version"].mask(laptops["os"] == "macOS","X")
# laptops["os_version"] = laptops["os_version"].mask(laptops["os"] == "No OS","Version Unknown")

In [33]:
value_counts_after = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()
value_counts_after

Linux        62
Chrome OS    27
Android       2
Name: os, dtype: int64

We'll clean the weight column now

In [34]:
laptops["weight"] = laptops["weight"].str.replace("kg","").str.replace("s","").astype(float)
laptops["weight"].head()

0    1.37
1    1.34
2    1.86
3    1.83
4    1.37
Name: weight, dtype: float64

In [35]:
laptops.rename({"weight":"weight_kg"},inplace = True, axis = 1)


In [36]:
laptops.columns

Index(['manufacturer', 'model_name', 'category', 'screen_size_inches',
       'screen', 'cpu', 'ram_gb', 'storage', 'gpu', 'os', 'os_version',
       'weight_kg', 'price_euros', 'gpu_manufacturer', 'cpu_manufacturer'],
      dtype='object')

In [44]:
laptops.to_csv("laptops_cleaned.csv", index = False)

By default, pandas will save the index labels as a column in the CSV file. Our data set has integer labels that don't contain any data, so we don't need to save the index.

In [45]:
pd.read_csv("laptops_cleaned.csv").head(2)

,manufacturer,model_name,category,screen_size_inches,screen,cpu,ram_gb,storage,gpu,os,os_version,weight_kg,price_euros,gpu_manufacturer,cpu_manufacturer
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,X,1.37,"1339,69",Intel,Intel
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,X,1.34,"898,94",Intel,Intel
